# Enhancing Singapore Airlines' Service Through Automated Sentiment Analysis of Customer Reviews



**Motivation**



## Singapore Airlines Customer Reviews Dataset Information

The [Singapore Airlines Customer Reviews Dataset](https://www.kaggle.com/datasets/kanchana1990/singapore-airlines-reviews) aggregates 10,000 anonymized customer reviews, providing a broad perspective on the passenger experience with Singapore Airlines. 

The dimensions are shown below:
- **`published_date`**: Date and time of review publication.
- **`published_platform`**: Platform where the review was posted.
- **`rating`**: Customer satisfaction rating, from 1 (lowest) to 5 (highest).
- **`type`**: Specifies the content as a review.
- **`text`**: Detailed customer feedback.
- **`title`**: Summary of the review.
- **`helpful_votes`**: Number of users finding the review helpful.

## Additional web scraping of online reviews

During our EDA, we noticed two main trends in the distribution of our dataset:
1. Less than 10% of our reviews were published from the years 2022 to 2024, making it hard for us to capture recent trends in sentiment.
2. Most of the reviews were highly positive, which could mean that SIA had mostly positive reviews, nevertheless we wanted to get more information on negative reviews to improve the robustness of our model.

### TripAdvisor

We scraped more data for airline reviews from TripAdvisor, specifically for the years 2022 to 2024. 
(https://www.tripadvisor.com.sg/Airline_Review-d8729151-Reviews-Singapore-Airlines)

The dimensions are shown below:
- **`Year`**: Year of review publication.
- **`Month`**: Month of review publication.
- **`Title`**: Title of review publication.
- **`Review Text`**: Main text content of review publication.
- **`Rating`**: Numerical rating provided by reviewer (Scale: 1 to 5)


### Skytrax

We also scraped from Skytrax, which is another data source for online reviews. 
(https://www.airlinequality.com/airline-reviews/singapore-airlines/?sortby=post_date%3ADesc&pagesize=100)

The dimensions are shown below:
- **`Year`**: Year of review publication.
- **`Month`**: Month of review publication.
- **`Title`**: Title of review publication.
- **`Review Text`**: Main text content of review publication.
- **`Rating`**: Numerical rating provided by reviewer (Scale: 1 to 10)

## Importing Libraries

Please uncomment the code box below to pip install relevant dependencies for this notebook.

In [1]:
#!pip3 install -r requirements.txt

In [9]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np
from datetime import datetime 

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm

# Plotting and Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Language Detection packages
# `langdetect` for detecting language
from langdetect import detect as langdetect_detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
# `langid` for an alternative language detection method
from langid import classify as langid_classify

# Text Preprocessing and NLP
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords

import nltk

# Tokenizing sentences/words
from nltk.corpus import wordnet

# Tokenizing sentences/words
from nltk.tokenize import word_tokenize
# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer
# Regular expressions for text pattern matching
import re

# Word Cloud generation
from wordcloud import WordCloud

# For generating n-grams
from nltk.util import ngrams
from collections import Counter

In [10]:
## for Mac users, might have to install this manually

# Ensure require NLTK data is downloaded
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify
[nltk_data]     failed: unable to get local issuer certificate
[nltk_data]   

False

## Data Preparation (Loading CSV)

Load the final CSV file into a dataframe named `data`.

In [11]:
data = pd.read_csv('../final_df.csv')

In [12]:
data.head()

,processed_full_review,sentiment,year,month
0,upper deck seat extremely comfortable service ...,Positive,2018,10
1,currently stick plane cgk sin flight suppose d...,Negative,2018,11
2,travel via changi airport uk australia flight ...,Positive,2018,11
3,take many singapore airline flight especially ...,Positive,2018,11
4,hour morning flight but crew manage great brea...,Positive,2018,11


In [13]:
data['year'].value_counts()

year
2019    5183
2018    2613
2022    1221
2023    1157
2020     902
2024     511
2021      94
Name: count, dtype: int64

# Hashing Vectorization
We applied several text preprocessing techniques to prepare the dataset:
- TF-IDF with N-grams: Captures word combinations (bigrams/trigrams) to represent more complex patterns in text.

- Hashing Vectorizer: A memory-efficient method to represent text as a fixed-size sparse vector.

- Latent Semantic Analysis (LSA): Used singular value decomposition (SVD) to reduce the dimensionality of the TF-IDF matrix while capturing important features

- We combined these text features (from TF-IDF, Hashing, and LSA) with other numeric features like year and month to create the final feature matrix.

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.decomposition import TruncatedSVD

# Assuming your data is in a dataframe called 'df'
# The text column is 'processed_full_review', and the target column is 'sentiment'

# Step 1: TF-IDF with Bigrams/Trigrams
tfidf_ngram_vectorizer = TfidfVectorizer(ngram_range=(2, 3), max_features=5000)
X_tfidf_ngram = tfidf_ngram_vectorizer.fit_transform(data['processed_full_review'])

# Convert TF-IDF matrix to DataFrame for readability
tfidf_ngram_df = pd.DataFrame(X_tfidf_ngram.toarray(), columns=tfidf_ngram_vectorizer.get_feature_names_out())

# Step 2: Hashing Vectorizer
hash_vectorizer = HashingVectorizer(n_features=5000, alternate_sign=False)
X_hash = hash_vectorizer.fit_transform(data['processed_full_review'])

# Step 3: LSA (Latent Semantic Analysis) via SVD
# Applying SVD on the TF-IDF matrix for dimensionality reduction
svd = TruncatedSVD(n_components=100)
X_lsa = svd.fit_transform(X_tfidf_ngram)

# Combine features if needed
X_combined = pd.concat([tfidf_ngram_df, pd.DataFrame(X_lsa)], axis=1)

# Step 4: Select Other Columns (Year, Month, etc.)
df_selected = data[['year', 'month']]  # Modify based on the features you want to select
X_combined = pd.concat([X_combined, df_selected], axis=1)

# Output the final dataframe for inspection
X_combined.head()


,able fly,able get,able sleep,absolutely fantastic,absolutely no,access lounge,actual flight,additional cost,address name,adelaide singapore,...,92,93,94,95,96,97,98,99,year,month
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.020396,-0.014576,0.009496,-0.035269,-0.009489,-0.023210,-0.021754,0.014295,2018,10
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002001,0.006666,-0.002058,-0.000754,0.008554,-0.010652,-0.005584,-0.013823,2018,11
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.028956,0.003982,0.012730,0.004910,-0.001690,0.017463,-0.008606,-0.002797,2018,11
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.016996,-0.025058,-0.011830,-0.012667,0.015872,-0.028323,-0.003796,0.004330,2018,11
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.025894,0.017449,-0.052794,-0.019275,-0.038557,-0.074113,-0.002253,0.019201,2018,11


# RF

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Assuming 'X_combined' is the feature matrix and 'data['sentiment']' is the target
X_combined.columns = X_combined.columns.map(str)
y = data['sentiment']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Initialize and train Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Detailed classification report
print(classification_report(y_test, y_pred))


Accuracy: 76.21%
              precision    recall  f1-score   support

    Negative       0.76      0.54      0.63       523
     Neutral       0.67      0.01      0.01       264
    Positive       0.76      0.96      0.85      1550

    accuracy                           0.76      2337
   macro avg       0.73      0.51      0.50      2337
weighted avg       0.75      0.76      0.71      2337



# RF with cross-validation

Cross-validation is a technique used to evaluate the performance of a model by splitting the data into multiple subsets, training the model on some subsets, and validating it on others.

In [16]:
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Ensure X_combined columns are strings
X_combined.columns = X_combined.columns.map(str)
y = data['sentiment']

# Split data into separate train/test sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Step 1: Cross-Validation on the training set only
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validation on the training set
cv_scores = cross_val_score(rf_classifier, X_train, y_train, cv=cv, scoring='accuracy')

# Print Cross-validation results
print(f"Cross-validation accuracy scores: {cv_scores}")
print(f"Mean cross-validation accuracy: {cv_scores.mean():.2f}")

# Step 2: Train on the full training set after cross-validation
rf_classifier.fit(X_train, y_train)

# Step 3: Evaluate on the separate test set
y_pred = rf_classifier.predict(X_test)

# Step 4: Classification report on test set
print(classification_report(y_test, y_pred))


Cross-validation accuracy scores: [0.78598181 0.7811664  0.7870519  0.78063135 0.77408994]
Mean cross-validation accuracy: 0.78
              precision    recall  f1-score   support

    Negative       0.76      0.54      0.63       523
     Neutral       0.67      0.01      0.01       264
    Positive       0.76      0.96      0.85      1550

    accuracy                           0.76      2337
   macro avg       0.73      0.51      0.50      2337
weighted avg       0.75      0.76      0.71      2337



# RF with Grid Search for Hyperparameter Tuning

Instead of using default hyperparameters, we perform a Grid Search to find the best combination of hyperparameters (such as n_estimators, max_depth, and min_samples_split). 



In [17]:
from sklearn.model_selection import GridSearchCV

# Ensure X_combined columns are strings
X_combined.columns = X_combined.columns.map(str)
y = data['sentiment']

# Split data into separate train/test sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Set up Grid Search
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the model with Grid Search
grid_search.fit(X_train, y_train)

# Get the best parameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best cross-validation accuracy: {best_score:.2f}")

# Use the best model for predictions
best_rf_classifier = grid_search.best_estimator_
y_pred = best_rf_classifier.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


Best parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Best cross-validation accuracy: 0.78
              precision    recall  f1-score   support

    Negative       0.76      0.55      0.64       523
     Neutral       0.75      0.01      0.02       264
    Positive       0.77      0.97      0.85      1550

    accuracy                           0.77      2337
   macro avg       0.76      0.51      0.51      2337
weighted avg       0.76      0.77      0.71      2337



# RF with Out-of-Bag (OOB) Evaluation

Random Forest has an internal method for evaluating performance called Out-of-Bag (OOB) score. It evaluates the model on samples not used during the training of individual trees, providing an internal cross-validation.

In [35]:
# Initialize Random Forest with OOB enabled
rf_classifier = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Print the OOB score
print(f"OOB Score: {rf_classifier.oob_score_:.2f}")

# Evaluate on the test set
y_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred))


OOB Score: 0.68
              precision    recall  f1-score   support

    Negative       0.62      0.97      0.75       178
     Neutral       0.00      0.00      0.00        38
    Positive       0.82      0.43      0.57       130

    accuracy                           0.66       346
   macro avg       0.48      0.47      0.44       346
weighted avg       0.63      0.66      0.60       346



/Users/ritikabajpai/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ritikabajpai/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ritikabajpai/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

# Ensemble Stacking

Stacking is a technique where you combine the predictions of multiple models, not just Random Forests, but any set of models, to make a final prediction.

In [37]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Define the base models
base_estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svm', SVC(kernel='linear', probability=True, random_state=42))
]

# Define the final estimator (meta-learner)
stacking_clf = StackingClassifier(estimators=base_estimators, final_estimator=LogisticRegression())

# Train the stacking classifier
stacking_clf.fit(X_train, y_train)

# Make predictions
y_pred = stacking_clf.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    Negative       0.73      0.90      0.81       178
     Neutral       0.00      0.00      0.00        38
    Positive       0.76      0.72      0.74       130

    accuracy                           0.73       346
   macro avg       0.49      0.54      0.51       346
weighted avg       0.66      0.73      0.69       346



# Log Regression

In [34]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42).fit(X_train, y_train)
clf_predictions = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, clf_predictions))
print("Classification Report:\n", classification_report(y_test, clf_predictions))

Accuracy: 0.7196531791907514
Classification Report:
               precision    recall  f1-score   support

    Negative       0.69      0.94      0.79       178
     Neutral       0.00      0.00      0.00        38
    Positive       0.80      0.62      0.70       130

    accuracy                           0.72       346
   macro avg       0.50      0.52      0.50       346
weighted avg       0.65      0.72      0.67       346



/Users/ritikabajpai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ritikabajpai/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ritikabajpai/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision 